In [ ]:
# Necessary imports remain the same
import os
import numpy as np
import cv2
import torch
from skimage import io, measure
from skimage.draw import polygon_perimeter
from scipy import ndimage as ndi
import matplotlib.pyplot as plt
from cellpose import models
import tifffile
from matplotlib.colors import ListedColormap
from ipywidgets import widgets, Layout, Button, Box, FloatText, FloatSlider, Textarea, Dropdown, Label, IntSlider, Text, Checkbox, VBox, HBox, Output, IntProgress
from IPython.display import display, HTML, clear_output
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
import time
import datetime # Added for timestamp

# --- Constants for Docker Environment ---
# These paths are relative to the container's root
INPUT_BASE_DIR = r'/app/input'
OUTPUT_BASE_DIR = r'/app/output'

# --- Functions (load_image, preprocess, detect_myotubes, analyze_myotubes, etc.) ---

def load_image(image_path):
    """Loads an image from the specified path."""
    return io.imread(image_path)

def preprocess(image, myotube_channel='green'):
    if image.ndim == 2:
        selected_channel = image
        blue_channel = image
    else:
        channel_idx = {'red': 0, 'green': 1}
        selected_channel = image[:, :, channel_idx[myotube_channel]]
        blue_channel = image[:, :, 2]  # blue channel for nuclei detection
    return selected_channel, blue_channel

def save_info_log(params_dict, log_messages, log_dir, output_widget=None):
    """Saves a text file with all parameters and log messages from the run."""
    def update_output(message):
        if output_widget is not None:
            with output_widget:
                display(HTML(f"<p>{message}</p>"))
        else:
            print(message)
    
    os.makedirs(log_dir, exist_ok=True)
    log_file_path = os.path.join(log_dir, "analysis_info_log.txt")
    
    try:
        with open(log_file_path, 'w') as f:
            # Write timestamp
            f.write(f"=== MYOTUBE ANALYSIS RUN - {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')} ===\n\n")
            
            # Write parameters section
            f.write("=== PARAMETERS ===\n")
            for param_name, param_value in params_dict.items():
                f.write(f"{param_name}: {param_value}\n")
            f.write("\n")
            
            # Write log messages section
            f.write("=== PROCESSING LOG ===\n")
            for msg in log_messages:
                # Strip HTML tags for cleaner text log
                clean_msg = msg.replace("<p>", "").replace("</p>", "").replace("<hr>", "\n---\n")
                clean_msg = clean_msg.replace("<b>", "").replace("</b>", "")
                f.write(f"{clean_msg}\n")
            
        update_output(f"Info log saved to: {log_file_path}")
    except Exception as e:
        update_output(f"<p style='color: red;'>Error saving info log: {e}</p>")

def detect_myotubes(green_channel, blue_channel, image, min_myotube_size, max_coverage_percent,
                    minimum_fiber_intensity, maximum_fiber_intensity, 
                    initial_nuclei_threshold, max_nuclei_threshold, 
                    nuclei_diameter, flow_threshold, cellprob_threshold, normalize,
                    output_widget=None):
    # Update output widget instead of print
    def update_output(message):
        if output_widget is not None:
            with output_widget:
                display(HTML(f"<p>{message}</p>"))
    
    update_output("Detecting myotubes and nuclei...")

    def _get_fiber_mask(fiber_channel, nuclei_channel, minimum_intensity, maximum_intensity, nuclei_threshold):
        # First, apply a base threshold
        kernel = np.ones((4, 4), np.uint8)
        _, min_thresh = cv2.threshold(fiber_channel, minimum_intensity, 255, cv2.THRESH_BINARY)
        _, max_thresh = cv2.threshold(fiber_channel, maximum_intensity, 255, cv2.THRESH_BINARY)
        processed = min_thresh - max_thresh
        
        # Opening, to remove noise in the background
        processed = cv2.morphologyEx(processed, cv2.MORPH_OPEN, kernel)
        
        # Closing, to remove noise inside the fibers
        processed = cv2.morphologyEx(processed, cv2.MORPH_CLOSE, kernel)
        
        # Inverting black and white
        processed = cv2.bitwise_not(processed)
        
        # Find contours of the holes inside the fibers
        contours, _ = cv2.findContours(processed, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        
        for contour in contours:
            mask = np.zeros_like(processed)
            cv2.drawContours(mask, [contour], -1, 255, -1)
            
            if np.average(nuclei_channel[mask == 255]) > nuclei_threshold:
                cv2.drawContours(processed, [contour], -1, 0, -1)
        
        # Inverting black and white again
        processed = cv2.bitwise_not(processed)
        
        # Creating a boolean mask
        mask = processed.astype(bool)
        
        return mask

    def calculate_coverage(mask):
        return np.sum(mask) / mask.size * 100

    # Detect nuclei using Cellpose
    update_output("Running Cellpose for nuclei detection...")
    
    # Use manual diameter if provided (nuclei_diameter > 0), otherwise set to None for auto-detection
    diameter_param = nuclei_diameter if nuclei_diameter > 0 else None
    if diameter_param:
        update_output(f"Using manual diameter: {diameter_param}")
    else:
        update_output("Using automatic diameter detection")
        
    model = models.Cellpose(model_type='nuclei', gpu=False)
    nuclei_masks, _, _, _ = model.eval(blue_channel, 
                                      diameter=diameter_param, 
                                      flow_threshold=flow_threshold, 
                                      cellprob_threshold=cellprob_threshold, 
                                      normalize=normalize, 
                                      channels=[0,0])
    
    # Create a binary mask for nuclei
    nuclei_binary = nuclei_masks > 0
    
    # Dilate nuclei slightly to ensure overlap with myotubes
    nuclei_dilated = cv2.dilate(nuclei_binary.astype(np.uint8), cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (9, 9)))

    nuclei_threshold = initial_nuclei_threshold
    
    update_output("Optimizing myotube detection...")
    while True:
        # Get the fiber mask
        myotube_mask = _get_fiber_mask(green_channel, blue_channel, minimum_fiber_intensity, maximum_fiber_intensity, nuclei_threshold)

        # Remove small objects
        myotube_mask = cv2.morphologyEx(myotube_mask.astype(np.uint8), cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5)))
        num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(myotube_mask, connectivity=8)
        for i in range(1, num_labels):
            if stats[i, cv2.CC_STAT_AREA] < min_myotube_size:
                myotube_mask[labels == i] = 0

        # Fill holes with overlapping nuclei
        overlapping_nuclei = nuclei_dilated & cv2.dilate(myotube_mask, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3)))
        myotube_mask = myotube_mask | overlapping_nuclei

        # Calculate coverage
        coverage = calculate_coverage(myotube_mask)
        
        update_output(f"Current nuclei threshold: {nuclei_threshold}, Coverage: {coverage:.2f}%")

        if coverage <= max_coverage_percent or nuclei_threshold >= max_nuclei_threshold:
            break
        
        nuclei_threshold += 20
        nuclei_threshold = min(nuclei_threshold, max_nuclei_threshold)

    # Label the myotubes
    labeled = measure.label(myotube_mask)
    
    update_output("Myotube detection complete.")
    return labeled, coverage, nuclei_masks

def analyze_myotubes(myotube_mask, nuclei_mask):
    # Get properties of labeled regions
    myotube_props = measure.regionprops(myotube_mask)
    
    # Label the nuclei
    labeled_nuclei = measure.label(nuclei_mask)
    
    myotube_data = []
    total_nuclei = np.max(labeled_nuclei)
    nuclei_in_myotubes = 0
    
    for prop in myotube_props:
        # Create a mask for the current myotube
        current_myotube_mask = myotube_mask == prop.label
        
        # Count nuclei inside the myotube
        nuclei_in_myotube = np.unique(labeled_nuclei[current_myotube_mask])
        nuclei_count = len(nuclei_in_myotube) - 1 if 0 in nuclei_in_myotube else len(nuclei_in_myotube)
        
        # Only process myotubes with at least 3 nuclei
        if nuclei_count >= 3:
            nuclei_in_myotubes += nuclei_count
            
            # Calculate length (major axis length)
            length = prop.major_axis_length
            
            # Calculate diameter (minor axis length)
            diameter = prop.minor_axis_length
            
            # Calculate surface area
            surface_area = prop.area
            
            # Append data for this myotube
            myotube_data.append({
                'id': prop.label,
                'nuclei_count': nuclei_count,
                'length': length,
                'diameter': diameter,
                'surface_area': surface_area
            })
    
    # Calculate Nuclear Fusion Index
    nuclear_fusion_index = (nuclei_in_myotubes / total_nuclei) * 100 if total_nuclei > 0 else 0
    
    return myotube_data, total_nuclei, nuclei_in_myotubes, nuclear_fusion_index

def create_colored_myotube_mask(myotube_mask):
    # Label each myotube uniquely
    labeled_myotubes = measure.label(myotube_mask)
    
    # Generate a colormap with unique colors for each label
    num_labels = np.max(labeled_myotubes)
    colors = plt.cm.get_cmap('tab20')(np.linspace(0, 1, num_labels + 1))
    colors[0] = [1, 1, 1, 1]  # Set background to white
    
    # Create a new colormap
    cmap = ListedColormap(colors)
    
    # Apply the colormap to the labeled myotubes
    colored_mask = cmap(labeled_myotubes)
    
    return (colored_mask * 255).astype(np.uint8)

def create_myotube_overlay(image, myotube_mask):
    # Create a colored overlay for myotubes
    overlay = image.copy()
    if overlay.ndim == 2:
        from skimage import color
        overlay = color.gray2rgb(overlay)
    
    # Find all contours using OpenCV, including internal ones
    contours, hierarchy = cv2.findContours(myotube_mask.astype(np.uint8), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    # Draw all contours on the overlay
    cv2.drawContours(overlay, contours, -1, (255, 0, 0), 2)
    
    return overlay

def create_nuclei_visualization(nuclei_mask, myotube_mask):
    # Create a visualization of nuclei
    nuclei_viz = np.zeros((*nuclei_mask.shape, 3), dtype=np.uint8)
    
    # Label nuclei
    labeled_nuclei = measure.label(nuclei_mask)
    
    # Identify nuclei inside myotubes
    nuclei_in_myotubes = labeled_nuclei * (myotube_mask > 0)
    
    # Color nuclei inside myotubes green, others red
    nuclei_viz[nuclei_in_myotubes > 0] = [0, 255, 0]  # Green
    nuclei_viz[(labeled_nuclei > 0) & (nuclei_in_myotubes == 0)] = [255, 0, 0]  # Red
    
    return nuclei_viz


def visualize_and_save_results(image, labeled_myotube_mask, labeled_nuclei_mask, coverage, image_name,
                               total_nuclei, nuclei_in_myotubes, nuclear_fusion_index,
                               save_dir=None, display_fig=True, output_widget=None):
    """Creates and optionally saves a multi-panel plot of the analysis results."""
    def update_output(message):
        if output_widget is not None:
            with output_widget:
                display(HTML(f"<p>{message}</p>"))
        else:
            print(message)

    update_output(f"Creating visualization for {image_name}...")

    fig, axes = plt.subplots(1, 4, figsize=(20, 5))
    fig.suptitle(f"Analysis Results for {image_name}", fontsize=16)
    ax1, ax2, ax3, ax4 = axes.ravel()

    # Original Image
    ax1.imshow(image, cmap='gray')
    ax1.set_title('Original Image')
    ax1.axis('off')

    # Colored Myotube Mask
    colored_myotube_mask = create_colored_myotube_mask(labeled_myotube_mask)
    ax2.imshow(colored_myotube_mask)
    ax2.set_title(f'Myotube Mask\n(Coverage: {coverage:.2f}%)')
    ax2.axis('off')

    # Myotube Overlay
    # Pass the *labeled* myotube mask to the overlay function
    overlay = create_myotube_overlay(image, labeled_myotube_mask)
    ax3.imshow(overlay)
    ax3.set_title('Myotube Overlay')
    ax3.axis('off')

    # Nuclei Visualization
    # Pass *labeled* masks to the nuclei visualization function
    nuclei_viz = create_nuclei_visualization(labeled_nuclei_mask, labeled_myotube_mask)
    ax4.imshow(nuclei_viz)
    ax4.set_title('Nuclei Visualization')
    ax4.axis('off')

    # Add statistics text to the last plot
    stats_text = (f'Total nuclei: {total_nuclei}\n'
                  f'Nuclei in myotubes: {nuclei_in_myotubes}\n'
                  f'Nuclear Fusion Index: {nuclear_fusion_index:.2f}%')
    ax4.text(0.02, 0.98, stats_text, fontsize=10, color='white', # Smaller font
             verticalalignment='top', bbox=dict(facecolor='black', alpha=0.6),
             transform=ax4.transAxes)

    plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Adjust layout to prevent title overlap

    # Display the plot in the output widget if requested
    if display_fig and output_widget is not None:
        with output_widget:
            display(fig) # Display the figure object directly

    # Save the figure if a save directory is provided
    if save_dir:
        # The save_dir should already be the specific 'plots' subfolder
        os.makedirs(save_dir, exist_ok=True) # Ensure it exists
        save_path = os.path.join(save_dir, f"{os.path.splitext(image_name)[0]}_analysis.png")
        try:
            fig.savefig(save_path, bbox_inches='tight', facecolor='white', edgecolor='none', dpi=150) # Set DPI
            update_output(f"Plot saved to: {save_path}")
        except Exception as e:
            update_output(f"<p style='color: red;'>Error saving plot {save_path}: {e}</p>")

    # IMPORTANT: Maybe i need to close the plot *after* saving and displaying to free memory,
    # If i always close, Voila might not show the last plot...
    # Let's keep it open for now, assuming Voila handles display correctly.
    # If memory becomes an issue for large datasets, i might need to close earlier plots.
    # plt.close(fig) # Keep open for Voila display


def save_mask_files(labeled_myotube_mask, labeled_nuclei_mask, save_dir, base_filename, output_widget=None):
    """Saves the labeled myotube and nuclei masks as TIFF files."""
    def update_output(message):
        if output_widget is not None:
            with output_widget:
                display(HTML(f"<p>{message}</p>"))
        else:
            print(message)

    # Ensure the save directory exists (should be the specific 'masks' subfolder)
    os.makedirs(save_dir, exist_ok=True)

    # Save myotube mask (as labeled uint16)
    myotube_filename = os.path.join(save_dir, f"{base_filename}_myotubes_mask.tif")
    try:
        tifffile.imwrite(myotube_filename, labeled_myotube_mask.astype(np.uint16))
        update_output(f"Saved myotube mask to: {myotube_filename}")
    except Exception as e:
        update_output(f"<p style='color: red;'>Error saving myotube mask {myotube_filename}: {e}</p>")

    # Save nuclei mask (as labeled uint16)
    nuclei_filename = os.path.join(save_dir, f"{base_filename}_nuclei_mask.tif")
    try:
        tifffile.imwrite(nuclei_filename, labeled_nuclei_mask.astype(np.uint16))
        update_output(f"Saved nuclei mask to: {nuclei_filename}")
    except Exception as e:
        update_output(f"<p style='color: red;'>Error saving nuclei mask {nuclei_filename}: {e}</p>")

def process_single_image(image_path, min_myotube_size, max_coverage_percent,
                         minimum_fiber_intensity, maximum_fiber_intensity,
                         initial_nuclei_threshold, max_nuclei_threshold,
                         nuclei_diameter, flow_threshold, cellprob_threshold, normalize,
                         myotube_channel='green',
                         save_plots=False, plots_dir=None, # Will be the specific subfolder path
                         save_masks=False, masks_dir=None, # Will be the specific subfolder path
                         output_widget=None):
    """Processes a single image: load, preprocess, detect, analyze, visualize, save."""
    # I don`t need to collect logs here as this function passes its output_widget to other functions
    # which will in turn call update_output that's modified in the calling function
    def update_output(message):
        if output_widget is not None:
            with output_widget:
                display(HTML(f"<p>{message}</p>"))
        else:
            print(message)

    update_output(f"Processing image: {os.path.basename(image_path)}")
    results = { # Initialize results dict
        'myotube_results': [], 'coverage': 0, 'total_nuclei': 0,
        'nuclei_in_myotubes': 0, 'nuclear_fusion_index': 0
    }

    try:
        image = load_image(image_path)
        selected_channel, blue_channel = preprocess(image, myotube_channel)

        labeled_myotube_mask, coverage, labeled_nuclei_mask = detect_myotubes(
            selected_channel, blue_channel, image, 
            min_myotube_size, max_coverage_percent,
            minimum_fiber_intensity, maximum_fiber_intensity,
            initial_nuclei_threshold, max_nuclei_threshold,
            nuclei_diameter, flow_threshold, cellprob_threshold, normalize,
            output_widget
        )

        update_output("Analyzing myotubes...")
        myotube_results, total_nuclei, nuclei_in_myotubes, nuclear_fusion_index = analyze_myotubes(
            labeled_myotube_mask, labeled_nuclei_mask
        )

        update_output(f"Total nuclei found: {total_nuclei}")
        update_output(f"Nuclei inside myotubes (with at least 3 nuclei): {nuclei_in_myotubes}")
        update_output(f"Nuclear Fusion Index: {nuclear_fusion_index:.2f}%")
        image_name = os.path.basename(image_path)

        # Visualize (always display in output widget) and save if requested
        visualize_and_save_results(
            image, labeled_myotube_mask, labeled_nuclei_mask, coverage, image_name,
            total_nuclei, nuclei_in_myotubes, nuclear_fusion_index,
            plots_dir if save_plots else None, # Pass the specific plots subdir path
            True, # Always display in widget
            output_widget
        )

        # Save masks if requested
        if save_masks and masks_dir:
             save_mask_files(labeled_myotube_mask, labeled_nuclei_mask,
                             masks_dir, # Pass the specific masks subdir path
                             os.path.splitext(image_name)[0], output_widget)

        # Update results dictionary
        results.update({
            'myotube_results': myotube_results,
            'coverage': coverage,
            'total_nuclei': total_nuclei,
            'nuclei_in_myotubes': nuclei_in_myotubes,
            'nuclear_fusion_index': nuclear_fusion_index
        })
        update_output(f"Finished processing {image_name}")

    except FileNotFoundError:
        update_output(f"Error: Image file not found at {image_path}")
    except Exception as e:
        update_output(f"An error occurred processing {os.path.basename(image_path)}: {e}")
        # Optionally log the full traceback here for debugging
        import traceback
        with output_widget:
             display(HTML(f"<pre>{traceback.format_exc()}</pre>"))

    return results


def process_dataset(input_directory, min_myotube_size, max_coverage_percent,
                    minimum_fiber_intensity, maximum_fiber_intensity,
                    initial_nuclei_threshold, max_nuclei_threshold,
                    nuclei_diameter, flow_threshold, cellprob_threshold, normalize,
                    myotube_channel='green',
                    save_plots=False, plots_dir=None, # Specific subfolder path or None
                    save_masks=False, masks_dir=None, # Specific subfolder path or None
                    save_results=False, results_dir=None, # Specific subfolder path or None
                    results_format=None,
                    output_widget=None, progress_widget=None):
    """Processes all valid image files in a given directory."""
    # Collection for log messages
    log_messages = []
    
    def update_output(message):
        # Add to log collection
        log_messages.append(message)
        # Display as before
        if output_widget is not None:
            with output_widget:
                display(HTML(f"<p>{message}</p>"))
        else:
            print(message)

    # Input directory is now the full path to the selected folder (e.g., /app/input/Experiment1)
    # Output directories (plots_dir, masks_dir, results_dir) are the full paths to the subfolders
    # within the timestamped main output folder (e.g., /app/output/Experiment1_20231027_103000/plots)

    if not os.path.isdir(input_directory):
        update_output(f"Error: Input directory not found: {input_directory}")
        return [], log_messages

    image_files = [f for f in os.listdir(input_directory)
                   if f.lower().endswith(('.tif', '.tiff', '.jpg', '.jpeg', '.png'))
                   and os.path.isfile(os.path.join(input_directory, f))]

    all_results = []
    num_images = len(image_files)

    if num_images == 0:
        update_output(f"No image files found in {input_directory}.")
        return [], log_messages

    update_output(f"Found {num_images} images to process in {os.path.basename(input_directory)}.")
    start_time_total = time.time()

    # Setup progress bar
    if progress_widget is not None:
        progress_widget.max = num_images
        progress_widget.value = 0
        progress_widget.description = f'Processing 0/{num_images}'

    for idx, image_file in enumerate(image_files, start=1):
        image_path = os.path.join(input_directory, image_file)

        update_output(f"<hr><b>Processing image {idx}/{num_images}: {image_file}</b>")
        if progress_widget is not None:
             progress_widget.description = f'Processing {idx}/{num_images}'
        start_time_image = time.time()

        single_result = process_single_image(
            image_path, min_myotube_size, max_coverage_percent,
            minimum_fiber_intensity, maximum_fiber_intensity,
            initial_nuclei_threshold, max_nuclei_threshold,
            nuclei_diameter, flow_threshold, cellprob_threshold, normalize,
            myotube_channel=myotube_channel,
            save_plots=save_plots, plots_dir=plots_dir, # Pass the specific subfolder path
            save_masks=save_masks, masks_dir=masks_dir, # Pass the specific subfolder path
            output_widget=output_widget
        )

        elapsed_time_image = time.time() - start_time_image
        total_elapsed_time = time.time() - start_time_total
        estimated_total_time = (total_elapsed_time / idx) * num_images if idx > 0 else 0
        estimated_time_left = estimated_total_time - total_elapsed_time

        update_output(f"Time for {image_file}: {elapsed_time_image:.2f}s")
        update_output(f"Total elapsed: {total_elapsed_time:.2f}s | Est. left: {estimated_time_left:.2f}s")

        all_results.append({
            'image': image_file,
            **single_result # Unpack results from single image processing
        })

        # Update progress bar
        if progress_widget is not None:
            progress_widget.value = idx

    # Save summary results if requested
    if save_results and results_dir:
        save_results_to_file(all_results, results_dir, results_format, output_widget) # results_dir is the 'data' subfolder

    update_output("<hr><b>Processing completed for all images.</b>")
    if progress_widget is not None:
         progress_widget.bar_style = 'success'
         progress_widget.description = f'Completed {num_images} images'


    return all_results, log_messages

# print_all_results and print_summary_results can remain the same
def print_all_results(results, output_widget=None):
    output = ""
    for result in results:
        output += f"<h3>Image: {result['image']}</h3>"
        if 'coverage' in result: # Check if keys exist (in case of processing error)
            output += f"<p>Coverage: {result['coverage']:.2f}%</p>"
            output += f"<p>Total number of nuclei: {result['total_nuclei']}</p>"
            output += f"<p>Number of nuclei in myotubes (>=3 nuclei): {result['nuclei_in_myotubes']}</p>"
            output += f"<p>Nuclear Fusion Index: {result['nuclear_fusion_index']:.2f}%</p>"
            output += f"<p>Number of myotubes (>=3 nuclei): {len(result['myotube_results'])}</p>"

            # Create a table for myotube data
            if len(result['myotube_results']) > 0:
                output += "<table border='1' style='border-collapse: collapse; width: 100%; font-size: 0.9em;'>" # Smaller font
                output += "<tr style='background-color: #f2f2f2;'><th>Myotube ID</th><th>Nuclei count</th><th>Length (px)</th><th>Diameter (px)</th><th>Area (px^2)</th></tr>"

                for myotube in result['myotube_results']:
                    output += f"<tr><td>{myotube['id']}</td><td>{myotube['nuclei_count']}</td><td>{myotube['length']:.2f}</td><td>{myotube['diameter']:.2f}</td><td>{myotube['surface_area']:.2f}</td></tr>"

                output += "</table>"
            else:
                 output += "<p>No myotubes with >= 3 nuclei found.</p>"
        else:
             output += "<p>Processing failed for this image.</p>"
        output += "<hr>"

    if output_widget is not None:
        with output_widget:
            display(HTML("<h2>Detailed Results per Image</h2>"))
            display(HTML(output))
    return output


def print_summary_results(results, output_widget=None):
    valid_results = [r for r in results if 'coverage' in r] # Filter out failed results
    if not valid_results:
         output = "<h2>Summary Results</h2><p>No valid results to summarize.</p>"
         if output_widget is not None:
              with output_widget: display(HTML(output))
         return output

    avg_myotube_count = np.mean([len(r['myotube_results']) for r in valid_results])
    avg_coverage = np.mean([r['coverage'] for r in valid_results])
    avg_nuclear_fusion_index = np.mean([r['nuclear_fusion_index'] for r in valid_results])
    avg_total_nuclei = np.mean([r['total_nuclei'] for r in valid_results])
    avg_nuclei_in_myotubes = np.mean([r['nuclei_in_myotubes'] for r in valid_results])

    output = "<h2>Overall Summary Results</h2>"
    output += f"<p>Number of images processed successfully: {len(valid_results)}/{len(results)}</p>"
    output += f"<p>Average number of myotubes (>=3 nuclei) per image: {avg_myotube_count:.2f}</p>"
    output += f"<p>Average coverage per image: {avg_coverage:.2f}%</p>"
    output += f"<p>Average Nuclear Fusion Index per image: {avg_nuclear_fusion_index:.2f}%</p>"
    output += f"<p>Average total nuclei per image: {avg_total_nuclei:.2f}</p>"
    output += f"<p>Average nuclei in myotubes (>=3 nuclei) per image: {avg_nuclei_in_myotubes:.2f}</p>"

    if output_widget is not None:
        with output_widget:
            display(HTML(output))
    return output


def save_results_to_file(results, directory, file_format, output_widget=None):
    """Saves the aggregated results to an Excel or CSV file."""
    def update_output(message):
        if output_widget is not None:
            with output_widget:
                display(HTML(f"<p>{message}</p>"))
        else:
            print(message)

    update_output(f"Saving results to {file_format} file...")
    os.makedirs(directory, exist_ok=True) # Ensure data subfolder exists

    all_data = []
    valid_results = [r for r in results if 'coverage' in r] # Use only valid results

    # --- Add Overall Summary Row ---
    if valid_results:
        avg_myotube_count = np.mean([len(r['myotube_results']) for r in valid_results])
        avg_coverage = np.mean([r['coverage'] for r in valid_results])
        avg_nfi = np.mean([r['nuclear_fusion_index'] for r in valid_results])
        avg_total_nuc = np.mean([r['total_nuclei'] for r in valid_results])
        avg_nuc_in_myo = np.mean([r['nuclei_in_myotubes'] for r in valid_results])
        avg_len = np.mean([m['length'] for r in valid_results for m in r['myotube_results']]) if any(r['myotube_results'] for r in valid_results) else 0
        avg_diam = np.mean([m['diameter'] for r in valid_results for m in r['myotube_results']]) if any(r['myotube_results'] for r in valid_results) else 0
        avg_area = np.mean([m['surface_area'] for r in valid_results for m in r['myotube_results']]) if any(r['myotube_results'] for r in valid_results) else 0
        avg_nuc_per_myo = np.mean([m['nuclei_count'] for r in valid_results for m in r['myotube_results']]) if any(r['myotube_results'] for r in valid_results) else 0

        all_data.append({
            'Row Type': 'Overall Summary', 'Image': f'{len(valid_results)} images', 'Myotube ID': 'Averages',
            'Length (px)': f'{avg_len:.2f}', 'Diameter (px)': f'{avg_diam:.2f}',
            'Area (px^2)': f'{avg_area:.2f}', 'Nuclei Count': f'{avg_nuc_per_myo:.2f}',
            'Coverage (%)': f'{avg_coverage:.2f}', 'Total Nuclei': f'{avg_total_nuc:.2f}',
            'Nuclei in Myotubes': f'{avg_nuc_in_myo:.2f}', 'Nuclear Fusion Index (%)': f'{avg_nfi:.2f}'
        })
        # Add a spacer row
        all_data.append({col: '' for col in all_data[0].keys()})


    # --- Add Image Summaries and Individual Myotube Data ---
    for result in valid_results:
        image_name = result['image']
        coverage = result['coverage']
        total_nuclei = result['total_nuclei']
        nuclei_in_myotubes = result['nuclei_in_myotubes']
        nuclear_fusion_index = result['nuclear_fusion_index']

        # Calculate averages for *this image*
        myotubes_in_image = result['myotube_results']
        avg_length = np.mean([m['length'] for m in myotubes_in_image]) if myotubes_in_image else 0
        avg_diameter = np.mean([m['diameter'] for m in myotubes_in_image]) if myotubes_in_image else 0
        avg_surface_area = np.mean([m['surface_area'] for m in myotubes_in_image]) if myotubes_in_image else 0
        avg_nuclei_count = np.mean([m['nuclei_count'] for m in myotubes_in_image]) if myotubes_in_image else 0

        # Add summary row for *this image*
        all_data.append({
            'Row Type': 'Image Summary', 'Image': image_name, 'Myotube ID': f'Summary ({len(myotubes_in_image)} myotubes)',
            'Length (px)': f'Avg: {avg_length:.2f}', 'Diameter (px)': f'Avg: {avg_diameter:.2f}',
            'Area (px^2)': f'Avg: {avg_surface_area:.2f}', 'Nuclei Count': f'Avg: {avg_nuclei_count:.2f}',
            'Coverage (%)': coverage, 'Total Nuclei': total_nuclei,
            'Nuclei in Myotubes': nuclei_in_myotubes, 'Nuclear Fusion Index (%)': nuclear_fusion_index
        })

        # Add individual myotube rows for *this image*
        for myotube in myotubes_in_image:
            all_data.append({
                'Row Type': 'Data', 'Image': image_name, 'Myotube ID': myotube['id'],
                'Length (px)': f"{myotube['length']:.2f}", # Format as string for consistency
                'Diameter (px)': f"{myotube['diameter']:.2f}",
                'Area (px^2)': f"{myotube['surface_area']:.2f}",
                'Nuclei Count': myotube['nuclei_count'],
                'Coverage (%)': '', 'Total Nuclei': '', 'Nuclei in Myotubes': '', 'Nuclear Fusion Index (%)': ''
            })
        # Add a spacer row after each image's data
        all_data.append({col: '' for col in all_data[0].keys()})


    # Create DataFrame
    if not all_data:
         update_output("No data to save.")
         return

    df = pd.DataFrame(all_data)
    # Reorder columns for better readability
    cols_order = ['Row Type', 'Image', 'Myotube ID', 'Nuclei Count', 'Length (px)', 'Diameter (px)', 'Area (px^2)',
                   'Coverage (%)', 'Total Nuclei', 'Nuclei in Myotubes', 'Nuclear Fusion Index (%)']
    df = df[cols_order]


    # Determine file extension and save
    file_format_lower = file_format.lower() if file_format else 'excel' # Default to excel
    if file_format_lower == 'excel':
        file_extension = '.xlsx'
        engine = 'openpyxl'
    elif file_format_lower == 'csv':
        file_extension = '.csv'
        engine = None # Not needed for CSV
    else:
        update_output(f"Unsupported file format: {file_format}, defaulting to Excel")
        file_extension = '.xlsx'
        engine = 'openpyxl'
        file_format_lower = 'excel'

    filename = f"myotube_analysis_summary{file_extension}"
    filepath = os.path.join(directory, filename)

    try:
        if file_format_lower == 'excel':
            df.to_excel(filepath, index=False, engine=engine)

            # Apply conditional formatting (requires openpyxl)
            try:
                workbook = load_workbook(filepath)
                worksheet = workbook.active
                # Define fills
                overall_summary_fill = PatternFill(start_color='FFC0C0C0', end_color='FFC0C0C0', fill_type='solid') # Light Grey
                image_summary_fill = PatternFill(start_color='FFFFFF99', end_color='FFFFFF99', fill_type='solid') # Light Yellow

                # Apply fills based on 'Row Type' column (column A, index 1)
                for row_idx in range(2, worksheet.max_row + 1): # Skip header row
                    row_type_cell = worksheet.cell(row=row_idx, column=1)
                    fill_to_apply = None
                    if row_type_cell.value == 'Overall Summary':
                        fill_to_apply = overall_summary_fill
                    elif row_type_cell.value == 'Image Summary':
                        fill_to_apply = image_summary_fill

                    if fill_to_apply:
                        for col_idx in range(1, worksheet.max_column + 1):
                            worksheet.cell(row=row_idx, column=col_idx).fill = fill_to_apply

                # Auto-adjust column widths (optional, can be slow)
                # for col in worksheet.columns:
                #     max_length = 0
                #     column = col[0].column_letter # Get column letter
                #     for cell in col:
                #         try: # Necessary to avoid error on empty cells
                #             if len(str(cell.value)) > max_length:
                #                 max_length = len(cell.value)
                #         except:
                #             pass
                #     adjusted_width = (max_length + 2)
                #     worksheet.column_dimensions[column].width = adjusted_width


                workbook.save(filepath)
                update_output(f"Results saved and formatted in: {filepath}")
            except ImportError:
                update_output("Warning: 'openpyxl' not installed. Excel file saved without formatting.")
            except Exception as e_format:
                 update_output(f"<p style='color: orange;'>Warning: Could not apply Excel formatting: {e_format}. File saved without formatting.</p>")

        else:  # csv
            df.to_csv(filepath, index=False)
            update_output(f"Results saved to CSV: {filepath}")

    except Exception as e:
        update_output(f"<p style='color: red;'>An error occurred while saving the results file: {str(e)}</p>")
        # Fallback attempt to CSV if Excel failed
        if file_format_lower == 'excel':
            try:
                csv_filepath = os.path.join(directory, "myotube_analysis_summary_fallback.csv")
                df.to_csv(csv_filepath, index=False)
                update_output(f"Attempted fallback save to CSV: {csv_filepath}")
            except Exception as e_csv:
                update_output(f"<p style='color: red;'>Failed to save results as CSV fallback: {str(e_csv)}</p>")


def create_widgets():
    """Creates and arranges all the UI widgets."""

    # --- Define Base Directories ---
    input_base_dir = INPUT_BASE_DIR
    output_base_dir = OUTPUT_BASE_DIR

    # --- Output and Progress Widgets ---
    output = Output(
        layout=Layout(width='98%', height='450px', border='1px solid #ccc', overflow_y='auto', padding='5px')
    )
    progress = IntProgress(
        value=0, min=0, max=100, description='Progress:', bar_style='info',
        orientation='horizontal', layout=Layout(width='98%')
    )

    # --- Input Selection Widget ---
    # variable 'e_msg' to store potential error messages for cleaner handling
    e_msg = None
    try:
        if not os.path.exists(input_base_dir):
             raise FileNotFoundError(f"Base input directory '{input_base_dir}' not found.")
        input_options = sorted([d for d in os.listdir(input_base_dir) if os.path.isdir(os.path.join(input_base_dir, d))])
        if not input_options:
            input_options = ["No input folders found"]
            input_value = "No input folders found"
        else:
             input_value = input_options[0]
    except FileNotFoundError as e:
        input_options = ["Input directory error!"]
        input_value = "Input directory error!"
        e_msg = f"<p style='color: red;'>Error: {e}. Ensure it's correctly mounted in Docker.</p>"
    except Exception as e:
         input_options = [f"Error listing inputs!"]
         input_value = input_options[0]
         e_msg = f"<p style='color: red;'>Error listing input folders: {e}</p>"

    # Display error message in output if it occurred during input scanning
    if e_msg:
         with output:
             display(HTML(e_msg)) # Use IPython.display.HTML here because it's inside an Output widget

    input_directory_dropdown = Dropdown(
        options=input_options,
        value=input_value,
        description='Select Input Folder:',
        layout=Layout(width='auto'),
        style={'description_width': 'initial'},
        disabled=(input_value in ["No input folders found", "Input directory error!", "Error listing inputs!"]) # Disable if error
    )

    # --- Parameter Widgets ---
    myotube_channel_selector = Dropdown(description='Myotube Channel:', options=['green', 'red'], value='green', style={'description_width': 'initial'})
    min_myotube_size = IntSlider(description='Min Myotube Size (pixels):', min=50, max=1000, step=10, value=200, style={'description_width': 'initial'}, layout=Layout(width='95%'))
    max_coverage_percent = FloatText(description='Max Area Coverage Target (%):', value=40, min=5, max=95, step=1, style={'description_width': 'initial'})

    minimum_fiber_intensity = IntSlider(description='Min Fiber Intensity:', min=0, max=254, step=1, value=25, style={'description_width': 'initial'}, layout=Layout(width='95%'))
    maximum_fiber_intensity = IntSlider(description='Max Fiber Intensity:', min=1, max=255, step=1, value=200, style={'description_width': 'initial'}, layout=Layout(width='95%'))
    initial_nuclei_threshold = IntSlider(description='Hole Fill Threshold (Initial Nuclei Intensity):', min=0, max=150, step=1, value=50, style={'description_width': 'initial'}, layout=Layout(width='95%'))
    max_nuclei_threshold = IntSlider(description='Hole Fill Threshold (Max Nuclei Intensity):', min=1, max=255, step=1, value=120, style={'description_width': 'initial'}, layout=Layout(width='95%'))

    # --- New Nuclei Detection Parameters ---
    nuclei_diameter = IntSlider(
        description='Nuclei Diameter (px, 0=auto):',
        min=0, max=100, step=1, value=0,
        style={'description_width': 'initial'},
        layout=Layout(width='95%')
    )
    
    flow_threshold = FloatSlider(
        description='Flow Threshold:',
        min=0.0, max=1.0, step=0.05, value=0.4,
        style={'description_width': 'initial'},
        layout=Layout(width='95%')
    )
    
    cellprob_threshold = FloatSlider(
        description='Cell Probability Threshold:',
        min=-6.0, max=6.0, step=0.1, value=0.0,
        style={'description_width': 'initial'},
        layout=Layout(width='95%')
    )
    
    normalize = Checkbox(
        description='Normalize Nuclei Images',
        value=True,
        indent=False
    )

    # --- Saving Options Widgets ---
    save_plots = Checkbox(description='Save Analysis Plots (.png)', value=True, indent=False)
    save_masks = Checkbox(description='Save Mask Files (.tif)', value=True, indent=False)
    save_results = Checkbox(description='Save Data Summary File', value=True, indent=False)
    results_format = Dropdown(description='Data File Format:', options=['Excel', 'CSV'], value='Excel', style={'description_width': 'initial'})

    # --- Run Button ---
    run_button = Button(description="Run Analysis", button_style='success', icon='play', layout=Layout(width='auto'))

    # --- Button Click Handler ---
    def on_button_click(b):
        # 1. Clear previous output and reset progress
        output.clear_output()
        # Display error message again if it exists from startup
        if e_msg:
            with output: display(HTML(e_msg)) # Use IPython.display.HTML here

        progress.value = 0
        progress.bar_style = 'info'
        progress.description = 'Starting...'

        with output:
            # Use IPython.display.HTML here because it's inside an Output widget
            display(HTML("<h2>Starting Analysis...</h2>"))
            display(HTML(f"<p><b>Input Folder:</b> {input_directory_dropdown.value}</p>"
                        f"<p><b>Myotube Channel:</b> {myotube_channel_selector.value}</p>"
                        f"<p><b>Min Myotube Size:</b> {min_myotube_size.value} | <b>Max Coverage Target:</b> {max_coverage_percent.value}%</p>"
                        f"<p><b>Intensity Thresholds:</b> Min={minimum_fiber_intensity.value}, Max={maximum_fiber_intensity.value}</p>"
                        f"<p><b>Hole Fill Thresholds:</b> Initial={initial_nuclei_threshold.value}, Max={max_nuclei_threshold.value}</p>"
                        f"<p><b>Nuclei Detection:</b> Diameter={nuclei_diameter.value} (0=auto), Flow={flow_threshold.value}, Cell Prob={cellprob_threshold.value}, Normalize={normalize.value}</p>"
                        f"<p><b>Save Options:</b> Plots={save_plots.value}, Masks={save_masks.value}, Data={save_results.value} ({results_format.value})</p>"
                        f"<hr>"))


        # 2. Validate Inputs
        selected_input_folder = input_directory_dropdown.value
        if selected_input_folder in ["No input folders found", "Input directory error!", "Error listing inputs!"]:
            with output:
                # Use IPython.display.HTML here
                display(HTML("<p style='color: red;'>Error: Please select a valid input folder from the dropdown.</p>"))
            progress.bar_style = 'danger'
            progress.description = 'Input Error'
            return

        if minimum_fiber_intensity.value >= maximum_fiber_intensity.value:
            with output:
                # Use IPython.display.HTML here
                display(HTML("<p style='color: orange;'>Warning: Min Fiber Intensity is >= Max Fiber Intensity. Results might be affected.</p>"))

        # 3. Construct Paths
        full_input_path = os.path.join(input_base_dir, selected_input_folder)

        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        main_output_dir = os.path.join(output_base_dir, f"{selected_input_folder}_{timestamp}")
        try:
            os.makedirs(main_output_dir, exist_ok=True)
            # Use IPython.display.HTML here
            with output: display(HTML(f"<p>Output will be saved to: {main_output_dir}</p>"))
        except Exception as e:
            with output:
                # Use IPython.display.HTML here
                display(HTML(f"<p style='color: red;'>Error creating output directory '{main_output_dir}': {e}. "
                            f"Ensure '{output_base_dir}' exists and has write permissions.</p>"))
            progress.bar_style = 'danger'
            progress.description = 'Output Error'
            return

        plots_dir = os.path.join(main_output_dir, 'plots') if save_plots.value else None
        masks_dir = os.path.join(main_output_dir, 'masks') if save_masks.value else None
        data_dir = os.path.join(main_output_dir, 'data') if save_results.value else None
        
        # Create parameters dictionary for logging
        params_dict = {
            'Input Directory': full_input_path,
            'Myotube Channel': myotube_channel_selector.value,
            'Min Myotube Size': min_myotube_size.value,
            'Max Coverage Target (%)': max_coverage_percent.value,
            'Min Fiber Intensity': minimum_fiber_intensity.value,
            'Max Fiber Intensity': maximum_fiber_intensity.value,
            'Initial Nuclei Threshold': initial_nuclei_threshold.value,
            'Max Nuclei Threshold': max_nuclei_threshold.value,
            'Nuclei Diameter': nuclei_diameter.value,
            'Flow Threshold': flow_threshold.value,
            'Cell Probability Threshold': cellprob_threshold.value,
            'Normalize Nuclei Images': normalize.value,
            'Save Plots': save_plots.value,
            'Save Masks': save_masks.value,
            'Save Results': save_results.value,
            'Results Format': results_format.value if save_results.value else 'N/A'
        }

        # 4. Run Processing
        try:
            # Modified to capture log messages
            results, log_messages = process_dataset(
                input_directory=full_input_path,
                min_myotube_size=min_myotube_size.value,
                max_coverage_percent=max_coverage_percent.value,
                minimum_fiber_intensity=minimum_fiber_intensity.value,
                maximum_fiber_intensity=maximum_fiber_intensity.value,
                initial_nuclei_threshold=initial_nuclei_threshold.value,
                max_nuclei_threshold=max_nuclei_threshold.value,
                nuclei_diameter=nuclei_diameter.value,
                flow_threshold=flow_threshold.value,
                cellprob_threshold=cellprob_threshold.value,
                normalize=normalize.value,
                myotube_channel=myotube_channel_selector.value,
                save_plots=save_plots.value,
                plots_dir=plots_dir,
                save_masks=save_masks.value,
                masks_dir=masks_dir,
                save_results=save_results.value,
                results_dir=data_dir,
                results_format=results_format.value if save_results.value else None,
                output_widget=output,
                progress_widget=progress
            )
            
            # Save info log file
            save_info_log(params_dict, log_messages, main_output_dir, output_widget=output)

            # 5. Display Summary
            with output:
                # Use IPython.display.HTML here
                display(HTML("<hr><h2>Analysis Finished</h2>"))
                if results is not None and results: # Check if processing returned any results (and not None)
                    print_summary_results(results, output_widget=output) # This function internally uses display(HTML(...))
                elif results is None: # Explicitly check for None, which might indicate an early exit or major error in process_dataset
                    display(HTML("<p style='color: red;'>Processing did not complete successfully. Check log above for errors.</p>"))
                else: # Empty list returned
                    display(HTML("<p>Processing completed, but no results were generated (e.g., no valid images found or processed).</p>"))

                if save_results.value and data_dir:
                    output_filename = f"myotube_analysis_summary.{'xlsx' if results_format.value == 'Excel' else 'csv'}"
                    full_output_path = os.path.join(data_dir, output_filename)
                    if os.path.exists(full_output_path): # Verify file was actually saved
                        display(HTML(f"<p><b>Data summary saved to:</b> {full_output_path}</p>"))
                    else:
                        display(HTML(f"<p style='color: orange;'><b>Warning:</b> Data summary file was expected but not found at {full_output_path}. Check logs for saving errors.</p>"))
                if save_plots.value and plots_dir:
                    display(HTML(f"<p><b>Plots saved in:</b> {plots_dir}</p>"))
                if save_masks.value and masks_dir:
                    display(HTML(f"<p><b>Masks saved in:</b> {masks_dir}</p>"))
                
                # Mention the info log file
                display(HTML(f"<p><b>Analysis log saved to:</b> {os.path.join(main_output_dir, 'analysis_info_log.txt')}</p>"))

        except Exception as e:
            with output:
                import traceback
                # Use IPython.display.HTML here
                display(HTML(f"<h2 style='color: red;'>Critical Error during Analysis</h2>"
                            f"<p>An unexpected error stopped the process:</p>"
                            f"<pre>{str(e)}</pre>"
                            f"<p>Traceback:</p>"
                            f"<pre>{traceback.format_exc()}</pre>"))
            progress.bar_style = 'danger'
            progress.description = 'Runtime Error!'

    run_button.on_click(on_button_click)

    # --- Arrange Widgets ---
    layout_box = Layout(border='1px solid #ddd', padding='10px', margin='5px 0px')

    input_section = Box([Label("1. Input Data Selection", style={'font_weight': 'bold'}), input_directory_dropdown], layout=layout_box)
    params_detection = Box([Label("2. Detection Parameters", style={'font_weight': 'bold'}), myotube_channel_selector, min_myotube_size, max_coverage_percent], layout=layout_box)
    params_thresholds = Box([Label("3. Intensity & Hole Filling Thresholds", style={'font_weight': 'bold'}), minimum_fiber_intensity, maximum_fiber_intensity, initial_nuclei_threshold, max_nuclei_threshold], layout=layout_box)
    
    # New section for nuclei detection parameters
    nuclei_params = Box([
        Label("4. Nuclei Detection Parameters", style={'font_weight': 'bold'}),
        nuclei_diameter,
        flow_threshold, 
        cellprob_threshold,
        normalize
    ], layout=layout_box)
    
    output_options = Box([Label("5. Output Options", style={'font_weight': 'bold'}), HBox([save_plots, save_masks, save_results]), results_format], layout=layout_box)


    # --- Main container VBox (Corrected) ---
    all_widgets = VBox([
        # Use widgets.HTML for static HTML elements in the layout
        widgets.HTML(f"""
            <div style="text-align: center; margin-bottom: 15px;">
                <svg viewBox="0 0 1200 250" xmlns="http://www.w3.org/2000/svg" style="max-width: 100%; height: auto;">
                    <!-- Background -->
                    <rect x="0" y="0" width="1200" height="250" rx="10" fill="#f8f9fa" />
                    <!-- Stylized myotube with nuclei - extended across header -->
                    <path d="M50,125 C150,95 250,85 350,90 C450,95 550,115 650,120 C750,125 850,115 950,110 C1050,105 1100,125 1150,125 C1100,145 1050,155 950,150 C850,145 750,135 650,140 C550,145 450,165 350,160 C250,155 150,145 50,125 Z" fill="#8bc34a" fill-opacity="0.7" stroke="#4caf50" stroke-width="2" />
                    <!-- Nuclei inside the myotube -->
                    <circle cx="200" cy="125" r="15" fill="#2c5aa0" />
                    <circle cx="400" cy="125" r="15" fill="#2c5aa0" />
                    <circle cx="600" cy="125" r="15" fill="#2c5aa0" />
                    <circle cx="800" cy="125" r="15" fill="#2c5aa0" />
                    <circle cx="1000" cy="125" r="15" fill="#2c5aa0" />
                    <!-- Targeting reticle over side nucleus -->
                    <circle cx="800" cy="125" r="30" fill="none" stroke="#ff5722" stroke-width="1.5" stroke-dasharray="5,3" />
                    <line x1="800" y1="85" x2="800" y2="165" stroke="#ff5722" stroke-width="1.5" />
                    <line x1="760" y1="125" x2="840" y2="125" stroke="#ff5722" stroke-width="1.5" />
                    <!-- MUSCLE Text - positioned above myotube -->
                    <text x="600" y="65" font-family="Arial, sans-serif" font-size="60" font-weight="bold" text-anchor="middle" fill="#2c5aa0">MUSCLE</text>
                    <!-- Subtitle text - larger and more prominent -->
                    <text x="600" y="190" font-family="Arial, sans-serif" font-size="24" font-weight="bold" text-anchor="middle" fill="#555">Myotube and Nuclei Skeletal Cell Locator &amp; Evaluator</text>
                </svg>
            </div>
        """),
        
        widgets.HTML(f"""
            <p>Select an input folder containing your images (TIFF, JPG, PNG). Adjust parameters and run the analysis.</p>
            <ul>
                <li>Input folders should be placed inside <code>{input_base_dir}</code> (mounted into the container).</li>
                <li>Output will be generated in timestamped subfolders within <code>{output_base_dir}</code> (mounted from the container).</li>
            </ul>
            <hr>
        """),
        input_section,
        params_detection,
        params_thresholds,
        nuclei_params,
        output_options,
        run_button,
        progress,
        # Use widgets.HTML here too
        widgets.HTML("<hr><h2>Analysis Log & Results</h2>"),
        output # The Output widget itself is a valid child widget
    ])

    # Display the final composed widget
    display(all_widgets)


# --- Run the Widget Creation Function ---
create_widgets()